In [12]:
import geopandas as gpd
import pandas as pd

In [1]:
import sys
sys.path.append('..')

In [2]:
import rsutils.s2_grid_utils

In [3]:
roi_filepath = '../../ethiopia/data/Ethiopia_Bounding_Boxs/Ethiopia_2021_East_Bounding_Box.shp'

In [7]:
east_bb_gdf = gpd.read_file(roi_filepath).to_crs('epsg:4326')

In [8]:
east_bb_gdf

,id,width,height,area,perimeter,geometry
0,0,95270.217295,98953.722659,9.427343e+09,388447.879909,"POLYGON ((38.5527 7.73211, 38.5517 8.62716, 39..."


In [9]:
east_bb_s2grids_gdf = rsutils.s2_grid_utils.get_s2_grids_gdf(geojson_epsg_4326=east_bb_gdf['geometry'][0])

In [15]:
pd.DataFrame(
    east_bb_s2grids_gdf[['id']].rename(columns={'id':'s2gridid'})
).to_csv('../data/ethiopia_east_bb_s2grids.csv', index=False)

In [16]:
datacube_catalog_gdf = gpd.read_file('../data/cluster_files/catalog.geojson')

In [19]:
east_bb_s2grids_gdf[
    ~east_bb_s2grids_gdf['id'].isin(datacube_catalog_gdf['roi_name'])
][['id']].rename(columns={'id':'s2gridid'}).to_csv('../data/failed_ethiopia_east_bb_s2grids.csv', index=False)